In [1]:
from rsam import RSAM

# Please change variables inside variables.py
from variables import *

# Plot RSAM

In [2]:
def main():
    # ============================================
    # Running RSAM per station
    for station in stations:

        csv_files[station]: list[str] = []

        # Calculate RSAM per station per date
        for date in dates:
            date_string = date.strftime('%Y-%m-%d')

            # Save stream on a list of streams
            streams[date_string]: Stream = RSAM.fill_streams(client, station, date)

            # Make sure stream is not empty
            if streams[date_string].count():

                # Run the RSAM per day
                rsam = RSAM(streams[date_string], resample=resample_rule, bands=bands)

                # Add RSAM CSV file result to CSV list 
                csv_files[station].append(rsam.save(rsam_directory))

        # Combining RSAM CSV results per station name
        print(RSAM.concatenate_csv(station, rsam_directory, resample_rule))
    
    # Plot RSAM
    RSAM.plot(stations, rsam_directory, axvspans=continuous_eruptions, axvlines=single_eruptions)

In [3]:
main()

D:\Projects\dsar\output\rsam\VG.PSAG.00.EHZ\combined_VG.PSAG.00.EHZ.csv
D:\Projects\dsar\output\rsam\VG.TMKS.00.EHZ\combined_VG.TMKS.00.EHZ.csv
